### Set up functions and routes

In [5]:
def get_time(location: str) -> str:
    """Useful to get the time in a specific location"""
    print(f"Result from: `get_time` function with location: `{location}`")
    return "get_time"


def get_news(category: str, country: str) -> str:
    """Useful to get the news in a specific country"""
    print(
        f"Result from: `get_news` function with category: `{category}` "
        f"and country: `{country}`"
    )
    return "get_news"

Now generate a dynamic routing config for each function

In [6]:
from semantic_router.route import Route, RouteConfig

functions = [get_time, get_news]
routes = []

for function in functions:
    route = await Route.from_dynamic_route(entity=function)
    routes.append(route)

route_config = RouteConfig(routes=routes)

2023-12-19 17:46:30 INFO semantic_router.utils.logger Generating dynamic route...
2023-12-19 17:46:40 INFO semantic_router.utils.logger Generated route config:
{
    "name": "get_time",
    "utterances": [
        "What's the time in New York?",
        "Can you tell me the time in Tokyo?",
        "What's the current time in London?",
        "Can you give me the time in Sydney?",
        "What's the time in Paris?"
    ]
}
2023-12-19 17:46:40 INFO semantic_router.utils.logger Generating dynamic route...
2023-12-19 17:46:43 INFO semantic_router.utils.logger Generated route config:
{
    "name": "get_news",
    "utterances": [
        "Tell me the latest news from the United States",
        "What's happening in India today?",
        "Can you give me the top stories from Japan",
        "Get me the breaking news from the UK",
        "What's the latest in Germany?"
    ]
}


In [7]:
# You can manually add or remove routes

get_weather_route = Route(
    name="get_weather",
    utterances=[
        "what is the weather in SF",
        "what is the current temperature in London?",
        "tomorrow's weather in Paris?",
    ],
)
route_config.add(get_weather_route)

route_config.remove("get_weather")

route_config.to_dict()

2023-12-19 17:46:43 INFO semantic_router.utils.logger Added route `get_weather`
2023-12-19 17:46:43 INFO semantic_router.utils.logger Removed route `get_weather`


[{'name': 'get_time',
  'utterances': ["What's the time in New York?",
   'Can you tell me the time in Tokyo?',
   "What's the current time in London?",
   'Can you give me the time in Sydney?',
   "What's the time in Paris?"],
  'description': None},
 {'name': 'get_news',
  'utterances': ['Tell me the latest news from the United States',
   "What's happening in India today?",
   'Can you give me the top stories from Japan',
   'Get me the breaking news from the UK',
   "What's the latest in Germany?"],
  'description': None}]

In [8]:
# Get a route by name
route_config.get("get_time")

Route(name='get_time', utterances=["What's the time in New York?", 'Can you tell me the time in Tokyo?', "What's the current time in London?", 'Can you give me the time in Sydney?', "What's the time in Paris?"], description=None)

Save config to a file (.json or .yaml)

In [9]:
route_config.to_file("route_config.json")

2023-12-19 17:46:43 INFO semantic_router.utils.logger Saving route config to route_config.json


### Define routing layer

Load from local file

In [10]:
from semantic_router.route import RouteConfig

route_config = RouteConfig.from_file("route_config.json")

2023-12-19 17:46:43 INFO semantic_router.utils.logger Loading route config from route_config.json


In [11]:
from semantic_router import RouteLayer

route_layer = RouteLayer(routes=route_config.routes)

Do a function call with functions as tool

In [12]:
from semantic_router.utils.function_call import route_and_execute

tools = [get_time, get_news]

await route_and_execute(
    query="What is the time in Stockholm?", functions=tools, route_layer=route_layer
)
await route_and_execute(
    query="What is the tech news in the Lithuania?",
    functions=tools,
    route_layer=route_layer,
)
await route_and_execute(query="Hi!", functions=tools, route_layer=route_layer)

2023-12-19 17:46:43 INFO semantic_router.utils.logger Extracting function input...


Calling function: get_time
Result from: `get_time` function with location: `Stockholm`


2023-12-19 17:46:49 INFO semantic_router.utils.logger Extracting function input...


Calling function: get_news
Result from: `get_news` function with category: `tech` and country: `Lithuania`


2023-12-19 17:46:52 WARNING semantic_router.utils.logger No function found, calling LLM...


'Hello! How can I assist you today?'